In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import product, combinations

In [26]:
projdir = '/u/home/t/terencew/project-cluo/igvf/pilot/multiome/'
donors = list(np.loadtxt(f'{projdir}/txt/donors.txt', dtype=str))
samples = list(np.loadtxt(f'{projdir}/txt/samples.txt', dtype=str))
s = samples[0]

gex_methods = ['demuxlet', 'freemuxlet', 'vireo', 'vireo_nogenos',
    'scsplit', 'scsplit_nogenos', 'souporcell', 'souporcell_nogenos', 'demuxalot']
atac_methods = ['demuxlet', 'freemuxlet',  'vireo', 'vireo_nogenos',
    'scsplit', 'scsplit_nogenos', 'souporcell', 'souporcell_nogenos', 'scavengers']

con_cols = ['C1', 'C2', 'I1', 'I2']
final_con_cols = ['C1', 'C2', 'I1', 'I2', 'donor']

pd_experiments = np.loadtxt(f'{projdir}/ambient/ambisim/prop_doub/txt/experiments.txt', dtype=str)
mux_experiments = np.loadtxt(f'{projdir}/ambient/ambisim/mux_test/txt/experiments.txt', dtype=str)

In [117]:
donor_combs = list(combinations(donors, 2))

In [3]:
gex_pd = pd.read_csv(f'{projdir}/csv/ambisim/prop_doub_gex.csv', sep='\t', index_col=0, header=0)
atac_pd = pd.read_csv(f'{projdir}/csv/ambisim/prop_doub_atac.csv', sep='\t', index_col=0, header=0)
gex_mux = pd.read_csv(f'{projdir}/csv/ambisim/mux_gex.csv', sep='\t', index_col=0, header=0)
atac_mux = pd.read_csv(f'{projdir}/csv/ambisim/mux_atac.csv', sep='\t', index_col=0, header=0)

In [4]:
column = 'demux_type'
truth_column = 'truth_type'
final_col = 'droplet_acc'
gex_pd[final_col] = gex_pd[column] == gex_pd[truth_column]
atac_pd[final_col] = atac_pd[column] == atac_pd[truth_column]
gex_mux[final_col] = gex_mux[column] == gex_mux[truth_column]
atac_mux[final_col] = atac_mux[column] == atac_mux[truth_column]

In [5]:
### new metric
column = 'donor_id'
truth_column = 'truth'
final_col = 'donor_acc'

gex_pd['donor_acc'] = True
atac_pd['donor_acc'] = True
gex_mux['donor_acc'] = True
atac_mux['donor_acc'] = True

mask = gex_pd['demux_type'] == 'singlet'
tmp = gex_pd[mask]
tmp[column] == tmp[truth_column]
gex_pd.loc[mask,final_col] = tmp[column] == tmp[truth_column]

mask = atac_pd['demux_type'] == 'singlet'
tmp = atac_pd[mask]
tmp[column] == tmp[truth_column]
atac_pd.loc[mask,final_col] = tmp[column] == tmp[truth_column]

mask = gex_mux['demux_type'] == 'singlet'
tmp = gex_mux[mask]
tmp[column] == tmp[truth_column]
gex_mux.loc[mask,final_col] = tmp[column] == tmp[truth_column]

mask = atac_mux['demux_type'] == 'singlet'
tmp = atac_mux[mask]
tmp[column] == tmp[truth_column]
atac_mux.loc[mask,final_col] = tmp[column] == tmp[truth_column]

### start with pd
- plot I1 rates

In [6]:
!ls {projdir}/ambient/ambisim/mux_test/{exp}/demux/merged/gex/{s}

ls: cannot access {projdir}/ambient/ambisim/mux_test/{exp}/demux/merged/gex/{s}: No such file or directory


In [7]:
!ls {indir}/gex

ls: cannot access {indir}/gex: No such file or directory


In [55]:
exp = '20_20'
cov = 0
indir = f'{projdir}/ambient/ambisim/prop_doub/{exp}/csv/var_consistency/'
gex_con = pd.read_csv(f'{indir}/gex/{s}/{exp}_cov{cov}_all.csv', sep='\t', header=0, index_col=0)
atac_con = pd.read_csv(f'{indir}/atac/{s}/{exp}_cov{cov}_all.csv', sep='\t', header=0, index_col=0)

In [56]:
!ls {projdir}/ambient/ambisim/prop_doub/{exp}/csv/demux/

individual_atac.csv  individual_gex.csv  singlet_atac.csv  singlet_gex.csv


In [57]:
indir = f'{projdir}/ambient/ambisim/prop_doub/'
gex_indiv = pd.read_csv(f'{indir}/{exp}/csv/demux/individual_gex.csv', sep='\t',
              header=0, index_col=0)
atac_indiv = pd.read_csv(f'{indir}/{exp}/csv/demux/individual_atac.csv', sep='\t',
              header=0, index_col=0)

In [58]:
def process_varcon(demux, varcon, donors, method):
    cols = ['C1', 'C2', 'I1', 'I2', 'donor']
    counts = pd.DataFrame(columns=cols)
    for donor in donors:
        bc_mask = demux[method] == donor
        tmp_assign = demux[bc_mask]
        tmp_con = varcon[bc_mask]
        col_mask = [donor in x for x in tmp_con.columns]
        select_cols = tmp_con.columns[col_mask]
        tmp_con = tmp_con[select_cols]
        tmp_con.columns = ['C1', 'C2', 'I1', 'I2']
        tmp_con['donor'] = donor
        counts = pd.concat([counts, tmp_con])
    return counts

In [122]:
def process_con(con):
    con['DP'] = np.sum(con.iloc[:,:4], axis=1)
    con['Consistent rate'] = (con['C1'] + con['C2']) / con['DP']
    con['Inconsistent rate'] = 1 - con['Consistent rate']
    con['C1 rate'] = con['C1'] / con['DP']
    con['C2 rate'] = con['C2'] / con['DP']
    con['I1 rate'] = con['I1'] / con['DP']
    con['I2 rate'] = con['I2'] / con['DP']
    return con

In [123]:
indir = f'{projdir}/ambient/ambisim/prop_doub/{exp}/csv/var_consistency/'
method = 'all'
gex_con = pd.read_csv(f'{indir}/gex/{s}/{exp}_cov{cov}_{method}.csv', sep='\t', header=0, index_col=0)

In [124]:
gex_con

,25_A4_C1,26_A10_C1,27_C4_C1,48_V3_C1,25_A4_C2,26_A10_C2,27_C4_C2,48_V3_C2,25_A4_I1,26_A10_I1,27_C4_I1,48_V3_I1,25_A4_I2,26_A10_I2,27_C4_I2,48_V3_I2
AAACAGCCAAACAACA-1_D0,4,97,5,3,1399,1711,1360,1441,369,57,408,327,18,18,19,18
AAACAGCCAAACATAG-1_D0,95,5,13,4,1727,1434,1399,1468,53,346,381,312,10,10,19,10
AAACAGCCAAACCCTA-1_D0,1,2,129,0,716,697,951,715,257,276,22,258,8,8,6,8
AAACAGCCAAACCTAT-1_D0,2,31,1,1,447,541,428,452,110,16,129,105,2,2,2,2
AAACAGCCAAACCTTG-1_D0,0,1,41,2,325,318,410,324,93,100,8,94,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AACAAGCCACTGACCG-1_D0,21,5,70,4,1098,1029,1107,1042,190,259,181,246,10,10,15,10
AACAAGCCACTGACTA-1_D0,14,2,2,18,472,433,395,471,52,91,129,53,5,5,5,5
AACAAGCCACTGGCCA-1_D0,1,2,47,30,589,586,677,671,189,192,101,107,3,3,3,3
AACAAGCCACTGGCTG-1_D0,2,10,3,6,364,404,341,394,79,39,102,49,3,3,3,3


### can I derive some sort of doublet filter with I1 rates?

In [ ]:
gex_doubs_

### I1 as a singleton-donor accuracy filter is not particularly useful

In [60]:
# indir = f'{projdir}/ambient/ambisim/prop_doub/{exp}/csv/var_consistency/'
# for method in gex_methods:
#     print(method)
#     gex_con = pd.read_csv(f'{indir}/gex/{s}/{exp}_cov{cov}_{method}.csv', sep='\t', header=0, index_col=0)

In [61]:
# indir = f'{projdir}/ambient/ambisim/prop_doub/{exp}/csv/var_consistency/'
# for method in atac_methods:
#     print(method)
#     atac_con = pd.read_csv(f'{indir}/atac/{s}/{exp}_cov{cov}_{method}.csv', sep='\t', header=0, index_col=0)

In [62]:
indir = f'{projdir}/ambient/ambisim/prop_doub/{exp}/csv/var_consistency/'
method = 'demuxlet'
gex_con = pd.read_csv(f'{indir}/gex/{s}/{exp}_cov{cov}_{method}.csv', sep='\t', header=0, index_col=0)
gex_con = process_con(gex_con)

In [63]:
gex_con

,C1,C2,I1,I2,donor,DP,Consistent rate,Inconsistent rate,C1 rate,C2 rate,I1 rate,I2 rate
AAACAGCCAAACATAG-1_D0,95,1727,53,10,25_A4,1885,0.966578,0.033422,0.050398,0.916180,0.028117,0.005305
AAACAGCCAAAGCCTC-1_D0,44,868,24,3,25_A4,939,0.971246,0.028754,0.046858,0.924388,0.025559,0.003195
AAACAGCCAAAGCGCA-1_D0,16,400,25,3,25_A4,444,0.936937,0.063063,0.036036,0.900901,0.056306,0.006757
AAACAGCCAAATTGCT-1_D0,17,307,3,0,25_A4,327,0.990826,0.009174,0.051988,0.938838,0.009174,0.000000
AAACAGCCAACCCTCC-1_D0,12,213,3,0,25_A4,228,0.986842,0.013158,0.052632,0.934211,0.013158,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
AACAAGCCAACATAAG-1_D0,28,721,63,2,48_V3,814,0.920147,0.079853,0.034398,0.885749,0.077396,0.002457
AACAAGCCAACTAGCC-1_D0,6,193,27,2,48_V3,228,0.872807,0.127193,0.026316,0.846491,0.118421,0.008772
AACAAGCCACAATACT-1_D0,14,398,52,5,48_V3,469,0.878465,0.121535,0.029851,0.848614,0.110874,0.010661
AACAAGCCACAGACTC-1_D0,17,587,68,2,48_V3,674,0.896142,0.103858,0.025223,0.870920,0.100890,0.002967


In [71]:
# c_mask = gex_con.index.isin(gex_indiv.index)
# d_mask = gex_indiv.index.isin(gex_con.index)

# final_con = gex_con[c_mask]
# final_demux = gex_indiv[d_mask][[method, 'truth']]
# final_demux = final_demux[final_demux['truth'] != 'doublet']

In [119]:
# final_demux

In [120]:
# np.sum(final_demux['demuxlet'] == final_demux['truth']) / final_demux.shape[0]